In [1]:
import wrds
import pandas as pd

In [2]:
db = wrds.Connection(wrds_username=[private], 
                     wrds_host=[private], 
                     port=[private], 
                     connect_args={'sslmode':'disable'})

Loading library list...
Done


In [3]:
latest_date = db.raw_sql("SELECT MAX(date) FROM crsp.dsf")
print(latest_date)

          max
0  2023-12-29


In [4]:
sp100_tickers = ['BIIB', 'BA', 'AXP', 'SLB', 'COP', 'AVGO', 'TMO', 'NEE', 'NKE', 'MO',
       'WBA', 'QCOM', 'COST', 'ACN', 'CVS', 'T', 'CVX', 'HD', 'DUK', 'CL',
       'MMM', 'CSCO', 'BAC', 'LOW', 'BLK', 'MDLZ', 'PM', 'UNH', 'VZ', 'CAT',
       'NVDA', 'FDX', 'RTX', 'AIG', 'TMUS', 'INTC', 'PEP', 'TGT', 'GD', 'GS',
       'MDT', 'IBM', 'DIS', 'ORCL', 'COF', 'MSFT', 'KO', 'BKNG', 'V', 'LLY',
       'ADBE', 'AMZN', 'SBUX', 'BMY', 'MRK', 'XOM', 'F', 'JNJ', 'USB', 'AMT',
       'EXC', 'AAPL', 'SPG', 'TXN', 'PFE', 'PG', 'LMT', 'MCD', 'NFLX', 'UNP',
       'HON',  'C', 'GOOG', 'AMGN', 'JPM', 'MA', 'CMCSA', 'ABT', 'SO',
       'GILD', 'MET', 'MS', 'EMR', 'UPS', 'CRM', 'DHR', 'GOOGL', 'GE', 'WFC',
       'WMT', 'ICSA', 'UMCSENT', 'HSN1F', 'UNRATE', 'HYBS']

start_date = '2010-01-01'
end_date = '2024-09-01'

prices_data = pd.DataFrame()

for ticker in sp100_tickers:
    data = db.raw_sql(f"""
        SELECT date, ret
        FROM crsp.dsf
        WHERE permno IN (SELECT permno FROM crsp.stocknames WHERE ticker = '{ticker}')
        AND date BETWEEN '{start_date}' AND '{end_date}'
    """)
    data['ticker'] = ticker
    prices_data = pd.concat([prices_data, data])

/tmp/ipykernel_1095235/1324414378.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prices_data = pd.concat([prices_data, data])
/tmp/ipykernel_1095235/1324414378.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prices_data = pd.concat([prices_data, data])
/tmp/ipykernel_1095235/1324414378.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the re

In [5]:
prices_data = prices_data[["date","ret","ticker"]]

In [6]:
prices_data["ret"].max()

0.439739

In [7]:
prices_data['date'] = pd.to_datetime(prices_data['date'], errors='coerce')
prices_data['date'] = prices_data['date'].dt.strftime('%Y-%m-%d')

In [8]:
prices_data = prices_data.drop_duplicates(subset=['date', 'ticker'])

In [9]:
prices_data =  prices_data.rename(columns={'date': 'Date'})

In [10]:
prices_data

,Date,ret,ticker
0,2010-01-04,0.002617,BIIB
1,2010-01-05,-0.004847,BIIB
2,2010-01-06,0.000937,BIIB
3,2010-01-07,-0.008235,BIIB
4,2010-01-08,0.020570,BIIB
...,...,...,...
3517,2023-12-22,0.011951,WMT
3518,2023-12-26,-0.001532,WMT
3519,2023-12-27,0.009398,WMT
3520,2023-12-28,-0.001964,WMT


In [11]:
SNP = prices_data.pivot(index='Date', columns='ticker', values='ret').reset_index()


In [12]:
SNP.fillna(method='ffill', inplace=True)

/tmp/ipykernel_1095235/3928346092.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  SNP.fillna(method='ffill', inplace=True)


In [13]:
SNP.isna().sum().sum()

0

In [14]:
SNP

ticker,Date,AAPL,ABT,ACN,ADBE,AIG,AMGN,AMT,AMZN,AVGO,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
0,2010-01-04,0.015555,0.008705,0.013735,0.008429,-0.003002,0.020246,0.005786,-0.004609,0.031165,...,0.034449,0.023318,0.014119,0.016437,0.007775,0.004528,0.015795,0.012227,0.014593,0.014078
1,2010-01-05,0.001729,-0.008079,0.006180,0.016447,-0.018735,-0.008582,0.016567,0.005900,0.007423,...,-0.001586,0.013917,0.001719,0.025350,-0.011459,0.001803,-0.008043,0.027452,-0.009958,0.003905
2,2010-01-06,-0.015906,0.005553,0.010631,-0.002122,-0.006478,-0.007515,0.005206,-0.018116,0.007895,...,0.009848,0.001508,-0.007378,0.010656,-0.013428,-0.028344,-0.007568,0.001425,-0.002235,0.008643
3,2010-01-07,-0.001849,0.008284,-0.000935,-0.019405,-0.019218,-0.009157,-0.002702,-0.017013,-0.006266,...,0.038377,-0.008886,-0.007606,0.019401,0.009307,-0.005952,0.005991,0.036286,0.000560,-0.003142
4,2010-01-08,0.006648,0.005112,-0.003978,-0.005422,0.026592,0.008886,0.002032,0.027077,0.007357,...,-0.009391,0.033886,0.048075,0.001655,0.002766,0.000630,0.001354,-0.009269,-0.005037,-0.004012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,2023-12-22,-0.005548,0.005070,0.016344,-0.002316,0.005408,0.017291,0.004247,-0.002730,-0.004710,...,0.000827,0.006072,0.006936,0.004618,-0.004277,0.001603,0.006140,-0.005460,0.011951,0.001769
3518,2023-12-26,-0.002841,0.001834,-0.002878,-0.000818,0.002988,-0.000915,0.004879,-0.000065,0.008833,...,-0.000538,0.007020,-0.001390,0.012871,0.002825,-0.000267,0.014874,0.009353,-0.001532,0.002257
3519,2023-12-27,0.000518,0.005127,0.001104,-0.003644,0.000298,0.009264,0.010081,-0.000456,-0.005053,...,0.005307,0.002120,-0.007530,-0.002950,-0.000888,-0.004536,0.001127,-0.006245,0.009398,-0.004699
3520,2023-12-28,0.002226,0.005556,-0.006303,-0.000939,0.010722,0.006736,0.002030,0.000261,-0.003339,...,0.004036,0.000854,0.002997,0.004097,0.005677,0.004824,-0.001501,0.003041,-0.001964,-0.014460


We added the macro variable (foucsd on Composite Leading Indicators Index) such as 
- ICSA : https://fred.stlouisfed.org/series/ICSA
- UMCSENT : https://fred.stlouisfed.org/series/UMCSENT
- HSN1F : https://fred.stlouisfed.org/series/HSN1F
- UNRATE : https://fred.stlouisfed.org/series/UNRATE
- HIgh-Yield Bond Spread : https://fred.stlouisfed.org/series/BAMLH0A0HYM2